In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#  Project Objective
Investigate whether there is any measurable difference in progress based on the eligibility category. There are three options:
*Preliminary Analysis*
+ Developmental evaluation (delay)
+ Diagnosed condition
+ Diagnosed condition, developmental evaluation (by both delay and diagnosis)
1. Start by looking at overall progress by eligibility category (column AI of the "Elig Timeline Rpt 2018-2022" tab)
2. Factor in the time of service ("ECO with Exit-21-22" tab)
3. Do the above comparison by POE as well (column A of the "ECO with Exit21-22")
*Additional Analysis*
Additional areas you can look into are listed below:
+ Does typical time of service differ for different eligibility categories?


In [ ]:
teis = pd.ExcelFile("../Data/TEIS_NSS_Project_Data_10_2022.xlsx")
eco_21_22_exit = pd.read_excel(teis, 'ECO with Exit21-22', nrows=8633)
elig = pd.read_excel(teis, 'Elig Timeline Rpt 2018-2022', nrows=67611)

In [ ]:
eco_21_22_exit["CHILD_ID"] = eco_21_22_exit["CHILD_ID"].fillna(0).apply(np.int64)
eco_21_22_exit

In [ ]:
print(elig['Init. Elig. Category'].value_counts()['Developmental Evaluation'])

In [ ]:
print(elig['Init. Elig. Category'].value_counts()['Diagnosed Condition'])

In [ ]:
print(elig['Init. Elig. Category'].value_counts()['Diagnosed Condition, Developmental Evaluation'])

In [ ]:
elig = elig[['District', 'Child ID', 'Init. Elig. Category']]

In [ ]:
eco_21_22_exit = eco_21_22_exit[['DISTRICT', 'CHILD_ID', 'ECO_Entry_DATE', 'Ent SOCIAL_SCALE', 'Ent KNOWLEDGE_SCALE', 'Ent APPROPRIATE_ACTION_SCALE', 'ECO_Exit_DATE', 'Exit SOCIAL_SCALE', 'Exit KNOWLEDGE_SCALE', 'Exit APPROPRIATE_ACTION_SCALE']]

In [ ]:
elig.rename(columns = {'District':'DISTRICT', 'Child ID':'CHILD_ID'}, inplace = True)

In [ ]:
eco_exit_elig = pd.merge(elig, eco_21_22_exit, how="inner", on=["DISTRICT", "CHILD_ID"])

In [ ]:
eco_exit_elig

Subtracted entry score from exit score to gauge progress

In [ ]:
eco_exit_elig['SOCIAL SCALE PROGRESS'] = eco_exit_elig['Exit SOCIAL_SCALE'] - eco_exit_elig['Ent SOCIAL_SCALE']
eco_exit_elig

In [ ]:
eco_exit_elig['KNOWLEDGE SCALE PROGRESS'] = eco_exit_elig['Exit KNOWLEDGE_SCALE'] - eco_exit_elig['Ent KNOWLEDGE_SCALE']

In [ ]:
eco_exit_elig['APPROPRIATE ACTION PROGRESS'] = eco_exit_elig['Exit APPROPRIATE_ACTION_SCALE'] - eco_exit_elig['Ent APPROPRIATE_ACTION_SCALE']

In [ ]:
eco_exit_elig

In [ ]:
eco_exit_elig= eco_exit_elig.drop(['Ent SOCIAL_SCALE', 'Ent KNOWLEDGE_SCALE', 'Ent APPROPRIATE_ACTION_SCALE', 'Exit SOCIAL_SCALE', 'Exit KNOWLEDGE_SCALE', 'Exit APPROPRIATE_ACTION_SCALE'], axis=1)

Found total number of days

In [ ]:
eco_exit_elig['TOTAL DAYS'] = (eco_exit_elig['ECO_Exit_DATE'] - eco_exit_elig['ECO_Entry_DATE']) / np.timedelta64(1, 'D')

In [ ]:
eco_exit_elig

Filtered out cases where total time was less than 6 months

In [ ]:
eco_exit_elig=eco_exit_elig[eco_exit_elig['TOTAL DAYS']>=180]

In [ ]:
eco_exit_elig

Created new dataframes for Developmental Evaluation, Diagnosed Conditions, and Both

In [ ]:
dev_eval=eco_exit_elig[eco_exit_elig['Init. Elig. Category']=='Developmental Evaluation']

In [ ]:
dev_eval

In [ ]:
diag_cond=eco_exit_elig[eco_exit_elig['Init. Elig. Category']=='Diagnosed Condition']

In [ ]:
diag_cond

In [ ]:
dev_diag=eco_exit_elig[eco_exit_elig['Init. Elig. Category']=='Diagnosed Condition, Developmental Evaluation']

In [ ]:
dev_diag

In [ ]:
dev_eval[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
diag_cond[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
dev_diag[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
eco_exit_elig

In [ ]:
eco_exit_elig.DISTRICT.unique()

In [ ]:
ET = eco_exit_elig[eco_exit_elig['DISTRICT']=='ET']
FT = eco_exit_elig[eco_exit_elig['DISTRICT']=='FT']
GN = eco_exit_elig[eco_exit_elig['DISTRICT']=='GN']
MD = eco_exit_elig[eco_exit_elig['DISTRICT']=='MD']
NW = eco_exit_elig[eco_exit_elig['DISTRICT']=='NW']
SC = eco_exit_elig[eco_exit_elig['DISTRICT']=='SC']
SW = eco_exit_elig[eco_exit_elig['DISTRICT']=='SW']
UC = eco_exit_elig[eco_exit_elig['DISTRICT']=='UC']

In [ ]:
ET[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
FT[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
GN[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
MD[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
NW[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
SC[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
SW[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()

In [ ]:
UC[["SOCIAL SCALE PROGRESS","KNOWLEDGE SCALE PROGRESS", "APPROPRIATE ACTION PROGRESS", "TOTAL DAYS"]].mean()